In [24]:
# [connect kaggle with google colab to get dataset without downloading it]

# kaggle information

import os
os.environ['KAGGLE_USERNAME'] = ''                                           # kaggle username
os.environ['KAGGLE_KEY']      = ''                                           #kaggle key

In [ ]:
# dataset path from kggle
!kaggle datasets download ashishjangra27/gender-detection-20k-images-celeba

Dataset URL: https://www.kaggle.com/datasets/ashishjangra27/gender-detection-20k-images-celeba
License(s): unknown
gender-detection-20k-images-celeba.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# unzip file
!unzip gender-detection-20k-images-celeba.zip

In [41]:
# Import Libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
from google.colab import files

In [ ]:
# Preparation of Data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen  = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen   = ImageDataGenerator(rescale=1./255)

In [ ]:
# Dataset paths
train_path = '/content/Gender Detection Tiny/Train'
test_path  = '/content/Gender Detection Tiny/Test'
val_path   = '/content/Gender Detection Tiny/Validation'

In [ ]:
# Read Data
train_data = train_datagen.flow_from_directory(train_path, target_size=(64, 64), batch_size=32, class_mode='binary')
test_data  = test_datagen.flow_from_directory(test_path, target_size=(64, 64), batch_size=32, class_mode='binary')
val_data   = val_datagen.flow_from_directory(val_path, target_size=(64, 64), batch_size=32, class_mode='binary')

Found 18000 images belonging to 2 classes.
Found 1001 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
print('indice:', train_data.class_indices)

indice: {'Female': 0, 'Male': 1}


In [31]:
# Build Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.5),

    Flatten(),

    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [32]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, steps_per_epoch=train_data.samples // train_data.batch_size, epochs=10,
                    validation_data=val_data, validation_steps=val_data.samples // val_data.batch_size)

Epoch 1/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 49s 80ms/step - accuracy: 0.6961 - loss: 0.5618 - val_accuracy: 0.9153 - val_loss: 0.2195
Epoch 2/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step - accuracy: 0.9688 - loss: 0.1489 - val_accuracy: 1.0000 - val_loss: 0.1257
Epoch 3/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 78s 77ms/step - accuracy: 0.8987 - loss: 0.2489 - val_accuracy: 0.9294 - val_loss: 0.1613
Epoch 4/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 0s 40us/step - accuracy: 0.8750 - loss: 0.2561 - val_accuracy: 1.0000 - val_loss: 0.0233
Epoch 5/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 81s 75ms/step - accuracy: 0.9141 - loss: 0.2142 - val_accuracy: 0.9375 - val_loss: 0.1598
Epoch 6/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 1s 978us/step - accuracy: 0.9375 - loss: 0.1399 - val_accuracy: 1.0000 - val_loss: 0.0343
Epoch 7/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 82s 76ms/step - accuracy: 0.9274 - loss: 0.1870 - val_accuracy: 0.9486 - val_loss: 0.1309
Epoch 8/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.8750 - loss: 0.1421 - v

In [33]:
# Evaluate the model
loss, accuracy_test = model.evaluate(test_data)
print(f'Accuracy: {accuracy_test * 100:.2f}%')

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - accuracy: 0.9227 - loss: 0.1761
Accuracy: 93.01%


In [42]:
# Download & Save Model
model.save('model.h5')
files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
# Predict Code Part
test_image = image.load_img('/content/img1.jpg', target_size=(64,64))
test_image =  image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = model.predict(test_image)

prediction = (result > 0.005).astype(int)

print(prediction)

if prediction[0][0] == 1:
  prediction = 'Male'

else:
  prediction = 'Female'

print('the prediction result is:', prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0]]
the prediction result is: Female
